<a href="https://colab.research.google.com/github/techakilan/python-datascience-training/blob/master/PythonBasicDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

### CSV from Google drive

##### Mount from google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##### Import csv file from google drive and create dataframe

In [ ]:
iris_df = pd.read_csv('/content/drive/MyDrive/colabs_data/Iris.csv')
iris_df

##### head()

In [ ]:
iris_df.head()

In [ ]:
iris_df.head(10)

##### tail()

In [ ]:
iris_df.tail()

In [ ]:
iris_df.tail(10)

##### Get info() of the Dataframe

In [ ]:
iris_df.info()